Perform clustering (hierarchical,K means clustering and DBSCAN) for the airlines data to obtain optimum number of clusters. Draw the inferences from the clusters obtained.

Data Description:

The file EastWestAirlinescontains information on passengers who belong to an airline’s frequent flier program. For each passenger the data include information on their mileage history and on different ways they accrued or spent miles in the last year. The goal is to try to identify clusters of passengers that have similar characteristics for the purpose of targeting different segments for different types of mileage offers

ID --Unique ID

Balance--Number of miles eligible for award travel

Qual_mile--Number of miles counted as qualifying for Topflight status

cc1_miles -- Number of miles earned with freq. flyer credit card in the past 12 months: cc2_miles -- Number of miles earned with Rewards credit card in the past 12 months: cc3_miles -- Number of miles earned with Small Business credit card in the past 12 months:

1 = under 5,000 2 = 5,000 - 10,000 3 = 10,001 - 25,000 4 = 25,001 - 50,000 5 = over 50,000

Bonus_miles--Number of miles earned from non-flight bonus transactions in the past 12 months

Bonus_trans--Number of non-flight bonus transactions in the past 12 months

Flight_miles_12mo--Number of flight miles in the past 12 months

Flight_trans_12--Number of flight transactions in the past 12 months

Days_since_enrolled--Number of days since enrolled in flier program

Award--whether that person had award flight (free flight) or not

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
Air = pd.read_csv("Copy of EastWestAirlines.csv")
Air.head()

In [ ]:
## import hie. clustering lib
import scipy.cluster.hierarchy as sch ## for dendrogram and ploting
from sklearn.cluster import AgglomerativeClustering

In [ ]:
## before calculatimg the distances we neeed to do standardization
## customized / user defined normalization function
## here we can standardized functions as well from  sklearn but to show you how we can write custom function

def norm_func(i):
  x = (i-i.min())/(i.max()-i.min())
  return x                                            # (xi - min)/range (feature scaling ), range=max-min

In [ ]:
## for our understanding
# (i-i.min())/(i-i.max()-i.max())
Air['Balance'].min() , Air['Balance'].max()

In [ ]:
(0-0)/(1704838-0)

In [ ]:
#Normalized Dataframe
Air.iloc[:,1:]

In [ ]:
## consider all row and from 1: col to all columns to normalize the values
df_norm = norm_func(Air.iloc[:,1:])
df_norm

In [ ]:
df_norm.isna().sum()

In [ ]:
df_norm.describe()

In [ ]:
sns.pairplot(df_norm)
plt.show()

In [ ]:
df_norm.plot(kind='box', subplots=True, layout=(5,3), figsize=(25,10))
plt.show()

In [ ]:
df_norm.corr()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
fig, ax = plt.subplots(figsize=(15,15))
sns.heatmap(df_norm.corr(), annot=True, linewidths=0.2, linecolor="black", fmt=".2f", ax=ax)
plt.show()

In [ ]:
## values get scale down then create dendrogram

dendrogram = sch.dendrogram(sch.linkage(df_norm, method = 'single'))   ##  here using single linkage method
plt.figure(figsize=(70, 35))

other methods

In [ ]:
!pip install scipy

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage

import numpy as np

In [ ]:
## for the complete linkage method i have concerted data in numpy array
df_array = df_norm.to_numpy()

In [ ]:
linkage_matrix = linkage(df_array, method='complete')

In [ ]:
# The dendrogram function is now called dendrogram1, so we use that instead.
dendrogram1 = dendrogram(linkage_matrix)

In [ ]:
dendrogram2 = dendrogram(linkage(df_norm, method = 'centroid'))

Create cluster, dont have mentioned that how many clusters we have to make so i am going with 3 clusters

In [ ]:
hc = AgglomerativeClustering(n_clusters=3, affinity = 'euclidean', linkage= 'single')
hc

In [ ]:
## now i have created the clusters now i want to see the values are in which groups/clusters
## for that we are going to use fit predict method
y_hc = hc.fit_predict(df_norm)
clusters=pd.DataFrame(y_hc, columns=['clusters'])    ## append thses values in data frame

In [ ]:
y_hc

In [ ]:
# points to clusters.
clusters

In [ ]:
Air['cluster_id'] = clusters
Air

In [ ]:
Air_1 = Air.sort_values("cluster_id")

In [ ]:
cluster_sort = Air_1.iloc[:, [0,12]]
cluster_sort

In [ ]:
cluster_sort['cluster_id'].sort_values()

In [ ]:
# how to count unique values
## the number of customers to which clusters
Air['cluster_id'].value_counts()

K-means clustering

In [ ]:
## import some lib.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
## we dont know the no of clusters , so lets use k-means and elbow method to choose this number of optimal clusters
from sklearn.cluster import KMeans

In [ ]:
!pip install scikit-learn

In [ ]:
from sklearn.preprocessing import StandardScaler    ## built in function for standardizati

In [ ]:
## Normalizatio / standardization function

scaler = StandardScaler()  ## scaler is an object name of standardscaler class ## object intilizer

In [ ]:
tr_df = scaler.fit_transform(Air.iloc[:, 1:])
tr_df

In [ ]:
## how to find optimum number of culsters
## k-means algorithm aims to choose centroids that minimum the inertia or within cluster sum of squares intertia
import matplotlib.pyplot as plt

In [ ]:
## inertia meansures the how well a data set was calculated by k-means
# it s calculated by measuring the distance between each data point and its centriod, squaring this distance
# and suming these squares across one cluster.
## a good model is one with low inertia and low number of clusters(k)

# to plot th eelbow method graph, we need to compute the WCSS (within cluster sum of squares)
# let say max no. of clusters could be 10
# as we are going to have 10 iterations we are going to write a for loop to create a list of 10 WCSS for the no of clusters.
wcss = []   ## whithin  the cluster sum of squares.intilize wcss and begin the loop

for i in range(1, 11):
  kmeans = KMeans(n_clusters=i,init='k-means++', max_iter=300,random_state=0)  ## max ier 300 is default   ## inti = is random itilizer method
  kmeans.fit(tr_df)  ## use fit method to fit the kmeans object to our scaled data frame
  wcss.append(kmeans.inertia_) ## another name for wcss is inertia.in wcss list we will apend all distance ## this is like p and t value model.p-value

plt.plot(range(1,11), wcss)
plt.title('elbow method')
plt.xlabel('no of clusters')
plt.ylabel('wcss')
plt.show()

In [ ]:
wcss

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=0)  ## pass no of clusters as 4
predict = kmeans.fit_predict(tr_df)

In [ ]:
predict

In [ ]:
clusters_new = KMeans(n_clusters=3, random_state=0)
new_predict=clusters_new.fit_predict(tr_df)

In [ ]:
clusters_new.labels_

In [ ]:
new_predict

In [ ]:
Air['cluster_id'] = new_predict
Air.head()

In [ ]:
## group by the cluster id
Air.groupby('cluster_id').agg(['mean']).reset_index()

In [ ]:
Air[Air['cluster_id']==1]

In [ ]:
Air[Air['cluster_id']==2]

In [ ]:
print(Air[Air['cluster_id']==0].shape[0])
print(Air[Air['cluster_id']==1].shape[1])

DB SCAN

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
Air

In [ ]:
Air.info()

In [ ]:
Air.describe()

In [ ]:
Air.isna()

In [ ]:
Air.isna().sum()

In [ ]:
## check for outliers
Air.boxplot()

In [ ]:
plt.figure(figsize=(50, 25))
Air.boxplot()
plt.show()
## huge amout of outliers are there

In [ ]:
!pip install sklearn

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sc = StandardScaler()
Air = sc.fit_transform(Air)
Air

In [ ]:
## build model
from sklearn.cluster import DBSCAN

In [ ]:
db1 = DBSCAN()   ## db1 is instance  # default eps=0.5 , min_samples = 5
see = db1.fit_predict(Air)   ## we will get labels for clusters, -1 = outliers
see

In [ ]:
db2 = DBSCAN(eps=1, min_samples=3)  ## we are trying some values
db2.fit_predict(Air)

In [ ]:
db3 = DBSCAN(eps=1, min_samples=7)  ## we are trying some values
db3.fit_predict(Air)

In [ ]:
Df = pd.DataFrame(Air)

In [ ]:

sns.pairplot(df) ## pair plot is scatter plot in seaborn scatter plot is pairplo

In [ ]:
## evaluvate your model with silhoutte score
silhouette_score(Air,db1.fit_predict(Air)

In [ ]:
silhouette_score(Air,db2.fit_predict(Air) )

In [ ]:
silhouette_score(Air,db3.fit_predict(Air) )

In [ ]:
db4 = DBSCAN(eps=2, min_samples=7)  ## we are trying some values
db4.fit_predict(Air)
silhouette_score(Air,db4.fit_predict(Air) )

In [ ]:
db5 = DBSCAN(eps=3, min_samples=8)  ## we are trying some values
y = db5.fit_predict(Air)
silhouette_score(Air,db5.fit_predict(Air) )      ### this is the so far best value i get *********

In [ ]:
db6 = DBSCAN(eps=4, min_samples=9)
db6.fit_predict(Air)

In [ ]:
silhouette_score(Air,db6.fit_predict(Air) )

In [ ]:
db7 = DBSCAN(eps=3, min_samples=12)  ## we are trying some values
y = db7.fit_predict(Air)
silhouette_score(Air,db7.fit_predict(Air) )      ### this is the so far best value i get *********

In [ ]:
df1 = pd.read_csv("Copy of EastWestAirlines.csv")

In [ ]:
df1['cluster'] = y
df1

In [ ]:
##  extract outliers
df1[df1['cluster'] == -1]